# **Search algorithms in finding the best public transport connection and solving travelling salesman problem with tabu search**

### Importing data from csv file

In [21]:
import pandas as pd
df = pd.read_csv("connection_graph.csv", sep=',', header=0);

C:\Users\karol\AppData\Local\Temp\ipykernel_22988\3805628871.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("connection_graph.csv", sep=',', header=0);


### Basic information about dataset

The connection_graph.csv file contains data processed from the timetable of public transport in Wrocław for March 1, 2023. The data was downloaded using information provided by the Wrocław City Hall.

In [22]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'company', 'line', 'departure_time',
       'arrival_time', 'start_stop', 'end_stop', 'start_stop_lat',
       'start_stop_lon', 'end_stop_lat', 'end_stop_lon'],
      dtype='object')

In [23]:
df2 = pd.DataFrame(df)
display(df2)

,Unnamed: 0.1,Unnamed: 0,company,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon
0,0,0,MPK Autobusy,A,20:00:00,20:01:00,KRZYKI,Sowia,51.074884,17.006569,51.073793,17.001845
1,1,1,MPK Autobusy,A,20:01:00,20:02:00,Sowia,Chłodna,51.073793,17.001845,51.075122,16.996671
2,2,2,MPK Autobusy,A,20:02:00,20:03:00,Chłodna,Wawrzyniaka,51.075122,16.996671,51.078074,16.998202
3,3,3,MPK Autobusy,A,20:03:00,20:05:00,Wawrzyniaka,Rymarska,51.078074,16.998202,51.079323,16.991258
4,4,4,MPK Autobusy,A,20:05:00,20:06:00,Rymarska,RACŁAWICKA,51.079323,16.991258,51.077395,16.983938
...,...,...,...,...,...,...,...,...,...,...,...,...
273466,281502,281502,DLA Kąty Wrocławskie,967,18:38:00,18:39:00,Smolec - Lipowa/pętla,Smolec - Dębowa (sklep),51.075005,16.884421,51.076932,16.881543
273467,281503,281503,DLA Kąty Wrocławskie,967,18:39:00,18:41:00,Smolec - Dębowa (sklep),Krzeptów - skrzy.,51.076932,16.881543,51.088752,16.875808
273468,281504,281504,DLA Kąty Wrocławskie,967,18:41:00,18:42:00,Krzeptów - skrzy.,Krzeptów - Boisko,51.088752,16.875808,51.090678,16.885584
273469,281505,281505,DLA Kąty Wrocławskie,967,18:42:00,18:43:00,Krzeptów - Boisko,Krzeptów - Dolina Krzeptowa,51.090678,16.885584,51.092268,16.892523


In [24]:
print("Number of stops in dataset: ", len(set(list(df['start_stop']))))

Number of stops in dataset:  934


In [25]:
print("Number of records: ",len(list(df['start_stop'])))

Number of records:  273471


List of stop names

In [26]:
stops = list(set(list(df['start_stop'])))
print(stops[:10])

['Dolmed', 'Widawa', 'Witelona', 'Ślęza - Dębowa', 'Sosnowiecka', 'Chińska', 'Jędrzejowska', 'Polkowicka', 'Bielany Wrocławskie - Kościół', 'Poczta Główna']


### Graph implementation
In order to work on search algorithms, data has to be represented as a graph first. Graph itself is implemented as a nested dictionary, where first key represents departure stop and its value is also a dictionary with keys being neighbouring stops and values representing all information about connections between those stops as a list of connections. Connection itself is represented as a list with following convention: *[line, departure time, arrival time]*. Class Graph has also an additional dictionary for latitudes and longitudes for each stop.

In [27]:
class Graph:
    
    def __init__(self, num_of_nodes, list_of_nodes):
        self.m_num_of_nodes = num_of_nodes
        self.m_adj_list = {node: {} for node in list_of_nodes}
        self.m_lat_lon = {}

    def add_edge(self, node1, node2, connection, node1_lat_lon):
        if node2 in self.m_adj_list[node1]:
            self.m_adj_list[node1][node2].append(connection)
        else: 
            self.m_adj_list[node1][node2]=[connection]
        
        if node1 not in self.m_lat_lon:
            self.m_lat_lon[node1] = node1_lat_lon

    def print_adj_list_graph(self):
        for key in self.m_adj_list.keys():
            print("node", key, ": ", self.m_adj_list[key])

    def print_lat_lon_list(self):
        for key in self.m_lat_lon.keys():
            print("node", key, ": ", self.m_lat_lon[key])         
            
    def print_adj_list_node(self,node_key):
        print("node", node_key, ": ", self.m_adj_list[node_key])     

Having graph implemented, we can convert our dataset to graph. Each node represents one stop with information about connections saved in a nested dictionary. Departure and arrival time are converted to integer for simplification of calculations. Both represents number of minutes that passed between midnight and respective time.

In [28]:
from datetime import datetime

wroclaw_graph = Graph(len(stops),stops)
   
for i in range(len(df.index)):

    obj_dep_time = datetime.strptime(df['departure_time'][i],'%H:%M:%S').time()
    obj_arr_time = datetime.strptime(df['arrival_time'][i],'%H:%M:%S').time()
    converted_dep_time = obj_dep_time.hour*60 + obj_dep_time.minute
    converted_arr_time = obj_arr_time.hour*60 + obj_arr_time.minute
        
    wroclaw_graph.add_edge(df['start_stop'][i],df['end_stop'][i],[df['line'][i], converted_dep_time,
       converted_arr_time],(df['start_stop_lat'][i],df['start_stop_lon'][i]))

Examples od data representation:

In [55]:
print("List of Tyrmanda stop neighbours: ", wroclaw_graph.m_adj_list["Tyrmanda"].keys())

List of Tyrmanda stop neighbours:  dict_keys(['Zagony', 'MIŃSKA (Rondo Rotm. Pileckiego)'])


In [57]:
print("Latitude and longitude of Tyrmanda stop: ", wroclaw_graph.m_lat_lon['Tyrmanda'])

Latitude and longitude of Tyrmanda stop:  (51.10282257, 16.94521253)


In [63]:
print("List of ten examplary connections between Tyrmanda and Zagony stops: \n", wroclaw_graph.m_adj_list["Tyrmanda"]["Zagony"][:10])

List of ten examplary connections between Tyrmanda and Zagony stops: 
 [[107, 318, 319], [107, 468, 469], [107, 438, 439], [107, 879, 880], [107, 1016, 1017], [107, 911, 912], [107, 1047, 1048], [107, 405, 406], [107, 941, 942], [107, 1086, 1087]]


## **Dijskra algorithm**

**Our first approach will be to find the shortest (quickest) connection from source stop to target stop using classic Dijskra Algorithm.**

Let's define helper functions first. In our algorithms infity will be represented as 10 000 000, since all appearing values will be much lower.

In [75]:
#function finding node with minimum distance from source from a set of visited but not completed nodes
def minDistanceNode(dist, visited_not_finished):
        
    min = 1e7
 
    for v in visited_not_finished:
        if dist[v] < min:
            min = dist[v]
            min_name = v
 
    return min_name

#function finding earliest connection between selected stops at a given time
#if there is no connection after given time returns -1
def earliestConnection(graph,u,v,arr_time_at_u):
    u_v_con_list = graph.m_adj_list[u][v]
    min_dep = 1440
    min_con = -1
    for con in u_v_con_list:
        if con[1]>=arr_time_at_u and con[1] < min_dep:
            min_dep = con[1]
            min_con = con
    
    return min_con      

In [76]:
earliestConnection(wroclaw_graph,"Kwidzyńska","Zacisze",720)

[116, 720, 721]

Printing achived shortest path can be achieved by recursively iterating through previous connections from target to source. Below there is an implementation of function that will serve such purpose. Result has to be read from the bottom to the top.

In [124]:
def printRoute(prev,target,src,dep_time):
        
        print("Final destination: ", target, " reached at ", prev[target][3]//60,":",prev[target][3]%60,"\n")

        curr_stop = target
        change=False

        while curr_stop != src:
            if change:
                print(message)    
                change=False
                
            print(prev[curr_stop][0],"-",prev[curr_stop][1],"-",prev[curr_stop][2]//60,":",prev[curr_stop][2]%60)

            if prev[curr_stop][1] != prev[prev[curr_stop][0]][1] and prev[prev[curr_stop][0]][1]!=None:
                message = "\n"+str(prev[curr_stop][0]) + " reached at "+ str(prev[prev[curr_stop][0]][3]//60)+":"+str(prev[prev[curr_stop][0]][3]%60) +" You have to take next bus/tram now!\n"
                change=True
                
            curr_stop = prev[curr_stop][0]

        print("You are starting from: ", src, "at: ", dep_time//60,":",dep_time%60)  

Finally the implementation of Dijskra algorithm itself. Cost function in this case will be time in minutes. Algorithm will try to minimize this time.

In [126]:
def DijkstraAlgorithm(graph, src, target, dep_time_minutes):
    
    '''
    Initialize dictionaries: 
        -first with distances (in minutes) from source to each node (initialized with infinity for each one),
        -second with empty lists to save information about previous node (parent) and connection taken from this node, following convention:
            [previous stop name, line, departure time, arrival time] 
        -third with information wheter the shortest path from source to node is already calculated.

    Additionally in set "visited_not_finished" store names of stops that were visited but not yet has certainly shortest path calculated.
    '''
    
    dist_dict = {node: 1e7 for node in list(graph.m_adj_list)}
    prev = {node: [] for node in list(graph.m_adj_list)}
    calc_finished = {node: False for node in list(graph.m_adj_list)}

    visited_not_finished = set()

    #initialaze information for source stop
    dist_dict[src] = 0
    prev[src] = [None, None, None, dep_time_minutes]  
    visited_not_finished.add(src)

    '''
    Loop through all nodes (in worst case scenario), 
    in each iteration one node obtains shortest path from source,
    algorithm stops while reaching target node.
    '''

    for i in range(graph.m_num_of_nodes):
 
        node = minDistanceNode(dist_dict, visited_not_finished)

        if node == target:
            
            return prev, dist_dict[target]   

            
        #mark calculation of the shortest path to node u as completed
        calc_finished[node] = True
        visited_not_finished.remove(node)
 
        #get the time of arrival at node u from previous connection   
        arr_time_at_node = prev[node][3]

        #visiting all of node neighbours and updating info about shortest path found
        for neighbour in list(graph.m_adj_list[node]):
           
            if calc_finished[neighbour] == False:
                
                connection = earliestConnection(graph,node,neighbour,arr_time_at_node)

                #stopping when there is no connection to this node anymore
                if connection ==-1: 
                    break

                else:

                    visited_not_finished.add(neighbour)
                    dist_through_node = dist_dict[node] + (connection[2]-arr_time_at_node)
                    
                    #updating information about previous node if better connection was found
                    if dist_dict[neighbour] > dist_through_node:
                        dist_dict[neighbour] = dist_through_node
                        prev[neighbour] = [node, connection[0], connection[1], connection[2]]   

In [127]:
import time 

def RunDijkstraAlgorithm(graph, src, target, departure_time):
    
    #convert given time to number of minutes that passed from midnight
    obj_dep_time = datetime.strptime(departure_time,'%H:%M:%S').time()
    dep_time_minutes = obj_dep_time.hour*60 + obj_dep_time.minute
    
    start = time.time()
    prev, cost_minutes = DijkstraAlgorithm(graph, src, target, dep_time_minutes)
    end = time.time()

    printRoute(prev,target,src,dep_time_minutes)    
    print("\nTime in minutes of shortest path from source to target stop: ", cost_minutes)
    print("\nTime of running algorithm: ", end - start)

In [128]:
RunDijkstraAlgorithm(wroclaw_graph, 'Tyrmanda', 'most Grunwaldzki', '12:00:00')

Final destination:  most Grunwaldzki  reached at  12 : 37 

Urząd Wojewódzki (Impart) - 4 - 12 : 36
GALERIA DOMINIKAŃSKA - 4 - 12 : 33
Świdnicka - 4 - 12 : 30
Zamkowa - 4 - 12 : 28
Narodowe Forum Muzyki - 4 - 12 : 27
pl. Legionów - 4 - 12 : 25
Kolejowa - 4 - 12 : 23
Grabiszyńska - 4 - 12 : 22
Pereca - 4 - 12 : 21
Stalowa - 4 - 12 : 19
pl. Srebrny - 4 - 12 : 18
Bzowa (Centrum Zajezdnia) - 4 - 12 : 17
Hutmen - 4 - 12 : 16
FAT - 4 - 12 : 15

FAT reached at 12:13 You have to take next bus/tram now!

Ostrowskiego - 107 - 12 : 12
Końcowa - 107 - 12 : 10
Krzemieniecka - 107 - 12 : 9
Trawowa - 107 - 12 : 8
Stanisławowska (W.K. Formaty) - 107 - 12 : 7
Muchobór Wielki - 107 - 12 : 6
Zagony - 107 - 12 : 5
Tyrmanda - 107 - 12 : 4
You are starting from:  Tyrmanda at:  12 : 0

Time in minutes of shortest path from source to target stop:  37

Time of running algorithm:  0.030267715454101562


In [150]:
RunDijkstraAlgorithm(wroclaw_graph, 'Tyrmanda', 'Rynek', '14:00:00')

Final destination:  Rynek  reached at  14 : 35 

Zamkowa - 33 - 14 : 33

Zamkowa reached at 14:30 You have to take next bus/tram now!

Narodowe Forum Muzyki - 4 - 14 : 29
pl. Legionów - 4 - 14 : 27
Kolejowa - 4 - 14 : 25
Grabiszyńska - 4 - 14 : 24
Pereca - 4 - 14 : 23
Stalowa - 4 - 14 : 21
pl. Srebrny - 4 - 14 : 20
Bzowa (Centrum Zajezdnia) - 4 - 14 : 19
Hutmen - 4 - 14 : 18
FAT - 4 - 14 : 17

FAT reached at 14:16 You have to take next bus/tram now!

Ostrowskiego - 107 - 14 : 14
Końcowa - 107 - 14 : 12
Krzemieniecka - 107 - 14 : 11
Trawowa - 107 - 14 : 10
Stanisławowska (W.K. Formaty) - 107 - 14 : 8

Stanisławowska (W.K. Formaty) reached at 14:5 You have to take next bus/tram now!

Muchobór Wielki - 132 - 14 : 4
Zagony - 132 - 14 : 3
Tyrmanda - 132 - 14 : 2
You are starting from:  Tyrmanda at:  14 : 0

Time in minutes of shortest path from source to target stop:  35

Time of running algorithm:  0.05472874641418457


## **Algorithm A***

Second approach will be to use A* algorithm. It is a heuristic extension of Dijskra's algorithm with additional path cost estimation. The A* algorithm is based on cost function optimization which is defined as:

$$f(curr,next)=g(curr,next)+h(next,end)$$

 Now our algorithm will not only look at the time cost while picking minimum distance node, but also at the euclidean distance between each node and target node. In order to follow the same metric, we will convert euclidean distance to the approximate time that a vehicle needs to cover calculated distance with an average speed of 23 km/h. This information will be saved in h(next,end) function. The algorithm is based on the operation of two lists: open - containing a list visited nodes but whose neighbors are not all visited and closed list - nodes whose all neighbors have already been checked.

**We will also modify A\* algorithm to be able to search for connection with the lowest number of changes on the route.** In this case h function will stay the same, but g function will represent both a significant cost of change and cost of time that the connection takes, so it will favouritize resonably quick paths. The difference will be slightly different interpretation of graph and manipulating with weights parameters in order to find balance between satisfying results and the time of running calculations.

Euclidean distances will be calculated using a function below. Having latitudes and longitudes for each stop, we can follow formula used in proffessional calculators. Equation below comes from National Hurricane Center website https://www.nhc.noaa.gov/gccalc.shtml, http://edwilliams.org/avform147.htm#Dist.

In [131]:
from math import cos, sin, asin, sqrt, radians

#function calculating distance between stops in kilometers
def euclideanDist(node1_lat_lon, node2_lat_lon):
    x1,y1=node1_lat_lon
    x2,y2=node2_lat_lon
    EARTH_RAD = 6371

    d=2*EARTH_RAD*asin(sqrt((sin((radians(x1-x2))/2))**2 + cos(radians(x1))*cos(radians(x2))*(sin((radians(y1-y2))/2))**2))

    return d

In [132]:
euclideanDist(wroclaw_graph.m_lat_lon['Tyrmanda'],wroclaw_graph.m_lat_lon['most Grunwaldzki'])

7.714420821712388

### A* Algorithm with time criterion

In [151]:
def AlgorithmAstarTime(graph, src, target, dep_time):

    '''

    Initialize three dictionaries corresponding to values of three functions:
            
            g - time cost between source and each node respectively
            h - approx time to cover euclidean distance between target and each node respectively
            f - sum of both values above

        Values of those functions are initialized with infinity for each node (here 1e7 as mentioned before),
        except for source, where 0 is provided.
        
    Additionaly initialize closed and open set and dictionary of previous connections as before.

    '''
   
    g_dict = {node: 1e7 for node in list(graph.m_adj_list)}
    h_dict = {node: 1e7 for node in list(graph.m_adj_list)}
    f_dict = {node: 1e7 for node in list(graph.m_adj_list)}

    prev = {node: [] for node in list(graph.m_adj_list)}

    g_dict[src] = 0
    h_dict[src] = 0
    f_dict[src] = g_dict[src]+h_dict[src]
    prev[src] = [None,None,None,dep_time]

    
    closed = set()
    open = set()
    open.add(src)

    

    while len(open) > 0:

        node = None
        node_cost = 1e7

        #find node with the lowest f distance function currently in an open set
        for test_node in open:
            if f_dict[test_node] < node_cost:
                node = test_node
                node_cost = f_dict[test_node]
        
        #finish if target node reached, return previous connections dict and path cost
        if node == target:
            return prev, g_dict[target] 
        
        open.remove(node)
        closed.add(node)

        arr_time_at_node = prev[node][3]

        #iterate through all node's neighbours and update cost functions if better connection was found

        for next_node in list(graph.m_adj_list[node]):

            connection = earliestConnection(graph,node,next_node,arr_time_at_node)

            if connection ==-1: break

            if next_node not in open and next_node not in closed:
                
                open.add(next_node)
                h_dict[next_node] = euclideanDist(graph.m_lat_lon[next_node],graph.m_lat_lon[target])*60/23
                g_dict[next_node] = g_dict[node] + (connection[2]-arr_time_at_node)
                f_dict[next_node] = h_dict[next_node] + g_dict[next_node]
                prev[next_node] = [node,connection[0],connection[1],connection[2]]

            else:    
                if g_dict[next_node] > g_dict[node] + (connection[2]-arr_time_at_node):
                    g_dict[next_node] = g_dict[node] + (connection[2]-arr_time_at_node)
                    f_dict[next_node] = h_dict[next_node] + g_dict[next_node]
                    prev[next_node] = [node,connection[0],connection[1],connection[2]]
                    if next_node in closed:
                        open.add(next_node)
                        closed.remove(next_node)

### A* Algorithm with number of changes criterion

In [164]:
def printRoute2(prev,target,src,dep_time_source):
        
        print("Final destination : ", target[0], " reached at ",  target[1]//60,":", target[1]%60,"\n")
            
        curr_stop = target
        change = False
        
        while curr_stop != src:

            if change:
                print(message)    
                change=False

            dep_time=prev[curr_stop[0]][curr_stop[1]][2] 
            print(prev[curr_stop[0]][curr_stop[1]][0][0],"-", prev[curr_stop[0]][curr_stop[1]][1], "-", dep_time//60, ":",dep_time%60)

            prev_stop=prev[curr_stop[0]][curr_stop[1]][0]
            if prev[curr_stop[0]][curr_stop[1]][1] != prev[prev_stop[0]][prev_stop[1]][1] and prev[prev_stop[0]][prev_stop[1]][1]!=None:
                message = "\n"+str(prev[curr_stop[0]][curr_stop[1]][0][0]) + " reached at "+ str(prev[curr_stop[0]][curr_stop[1]][0][1]//60)+":"+str(prev[curr_stop[0]][curr_stop[1]][0][1]%60) +" You have to take next bus/tram now!\n"
                change=True


            curr_stop = prev[curr_stop[0]][curr_stop[1]][0]
            
        print("You are starting from: ", src[0], "at: ", dep_time_source//60,":",dep_time_source%60)  

In [165]:
def AlgorithmAstarChanges(graph, src, target, dep_time):


    g_dict = {node: {} for node in list(graph.m_adj_list)}
    h_dict = {node: {} for node in list(graph.m_adj_list)}
    f_dict = {node: {} for node in list(graph.m_adj_list)}
    prev = {node: {} for node in list(graph.m_adj_list)}


    earliest_con = 1440

    for source_neighbour in list(graph.m_adj_list[src]):

        con = earliestConnection(graph,src,source_neighbour,dep_time)
        if con!=-1:
            if con[1]<earliest_con: 
                earliest_con=con[1]

    source = (src,earliest_con)

    g_dict[src][earliest_con] = 0
    h_dict[src][earliest_con] = 0
    f_dict[src][earliest_con] = g_dict[src][earliest_con]+h_dict[src][earliest_con]

    
    prev[src][earliest_con] = [None,None,earliest_con]

    closed = set()
    open = set()
    open.add(source)
   

    while len(open) > 0:

        node = None
        node_cost = 1e7

        for test_node in open:
            stop_name=test_node[0]
            arrival_time=test_node[1]
            if f_dict[stop_name][arrival_time] < node_cost:
                node = test_node
                node_cost = f_dict[stop_name][arrival_time] 

        if node[0] == target:
            return prev,node,source
       
        open.remove(node)
        closed.add(node)

        stop_name=node[0]
        current_time=node[1] #time that we arrived at this stop at

        for next_stop_name in list(graph.m_adj_list[stop_name]):
            
            for connection in graph.m_adj_list[stop_name][next_stop_name]:
                
                #connection = [df['line'][i], converted_dep_time, converted_arr_time]

                if connection[1]>=current_time:
                    
                    change = 0 
                    if (prev[stop_name][current_time][1] == connection[0] and current_time==connection[1]) or prev[stop_name][current_time][1]==None:
                        change = 0 
                    else:
                        change = 60

                    next_node = (next_stop_name,connection[2])    
                    arr_time_next_node = connection[2]

                    if next_node not in open and next_node not in closed:
                        
                        open.add(next_node)
                        
                        h_dict[next_stop_name][arr_time_next_node] = euclideanDist(graph.m_lat_lon[next_stop_name],graph.m_lat_lon[target])*60/23*10
                        g_dict[next_stop_name][arr_time_next_node] = g_dict[stop_name][current_time] +(arr_time_next_node-current_time)+ change
                        f_dict[next_stop_name][arr_time_next_node] = h_dict[next_stop_name][arr_time_next_node] + g_dict[next_stop_name][arr_time_next_node]
                        prev[next_stop_name][arr_time_next_node] = [node,connection[0],connection[1]]
                        #poprzedni przystanek (pełna nazwa z godziną pojawienia się na tymprzystanku),linia,godz odjazdu--godzprzyjazdu w NAZWIE!!
                        
                    else:  

                        if (g_dict[next_stop_name][arr_time_next_node] > (g_dict[stop_name][current_time] + (arr_time_next_node-current_time) + change)):
                                g_dict[next_stop_name][arr_time_next_node] = g_dict[stop_name][current_time] + (arr_time_next_node-current_time) + change
                                f_dict[next_stop_name][arr_time_next_node] = h_dict[next_stop_name][arr_time_next_node] + g_dict[next_stop_name][arr_time_next_node]
                                prev[next_stop_name][arr_time_next_node] = [node,connection[0],connection[1]]

                                if next_node in closed:
                                    open.add(next_node)
                                    closed.remove(next_node)

In [168]:
def changesCounter(prev,target,src):
        
        curr_stop = target
        change_count = 0
        
        while curr_stop != src:

            prev_stop=prev[curr_stop[0]][curr_stop[1]][0]
            if prev[curr_stop[0]][curr_stop[1]][1] != prev[prev_stop[0]][prev_stop[1]][1] and prev[prev_stop[0]][prev_stop[1]][1]!=None:
                change_count+=1

            curr_stop = prev[curr_stop[0]][curr_stop[1]][0]
            
        return change_count

### Running A* Algorithm for selected criterion

In [169]:
def runAStarAlgorithm(graph, src, target, criterion, departure_time):
    
    if criterion!="t" and criterion!="p":
        raise Exception("Wrong criterion, please provide either t (time) or p (number of changes)")
    else:
        #convert given time to number of minutes that passed from midnight
        obj_dep_time = datetime.strptime(departure_time,'%H:%M:%S').time()
        dep_time_minutes = obj_dep_time.hour*60 + obj_dep_time.minute

        start = time.time()
        if criterion=="t":
            prev, cost_time = AlgorithmAstarTime(graph, src, target, dep_time_minutes)
            end = time.time()  
            printRoute(prev,target,src,dep_time_minutes)  
            print("\nTime in minutes of shortest path from source to target stop: ", cost_time)
        else: #criterion=="p"
            prev,target_tuple,source_tuple = AlgorithmAstarChanges(graph, src, target, dep_time_minutes)
            end = time.time()  
            printRoute2(prev,target_tuple,source_tuple,dep_time_minutes)
            print("\nNumber of changes needed on path from source to target stop: ", changesCounter(prev,target_tuple,source_tuple))
    
        print("\nTime of running algorithm: ", end - start)
        

In [173]:
runAStarAlgorithm(wroclaw_graph, 'Tyrmanda', 'FAT','p', '12:00:00')

Final destination :  FAT  reached at  12 : 13 

Ostrowskiego - 107 - 12 : 12
Końcowa - 107 - 12 : 10
Krzemieniecka - 107 - 12 : 9
Trawowa - 107 - 12 : 8
Stanisławowska (W.K. Formaty) - 107 - 12 : 7
Muchobór Wielki - 107 - 12 : 6
Zagony - 107 - 12 : 5
Tyrmanda - 107 - 12 : 4
You are starting from:  Tyrmanda at:  12 : 0

Number of changes needed on path from source to target stop:  0

Time of running algorithm:  0.04505562782287598


In [175]:
import random
from copy import deepcopy

def randomPermutation(stops_list):
    return list(random.sample(stops_list, len(stops_list)))

#function crucial for Tabu search to compare costs of each solution
def calcCostforSolution(graph, stops_permutation, starting_time, source):

    cur_time=AlgorithmAstarTime(wroclaw_graph,source, stops_permutation[0],starting_time)[0][stops_permutation[0]][3]

    for i in range(len(stops_permutation)-1):
        cur_time=AlgorithmAstarTime(graph,stops_permutation[i],stops_permutation[i+1],cur_time)[0][stops_permutation[i+1]][3]

    cur_time=AlgorithmAstarTime(graph,stops_permutation[-1],source,cur_time)[0][source][3]

    return cur_time-starting_time    

#helper function to print result
def printSolution(graph, stops_permutation, starting_time, source):

    result_prev=AlgorithmAstarTime(graph,source,stops_permutation[0],starting_time)[0]
    printRoute(result_prev,stops_permutation[0],source,starting_time)
    cur_time=result_prev[stops_permutation[0]][3]
    
    for i in range(len(stops_permutation)-1):
        result_prev=AlgorithmAstarTime(graph,stops_permutation[i],stops_permutation[i+1],cur_time)[0]
        printRoute(result_prev,stops_permutation[i+1],stops_permutation[i],cur_time)
        cur_time=result_prev[stops_permutation[i+1]][3]

    result_prev=AlgorithmAstarTime(graph,stops_permutation[-1],source,cur_time)[0]
    printRoute(result_prev,source,stops_permutation[-1],cur_time)
     

def travellingSalesmanUsingTabuSearch(graph,source,stops,starting_time,criterion="t"):
    k=0
    STEP_LIMIT=100

    T=set()
    starting_solution = randomPermutation(stops)
    best_known_solution = starting_solution
    best_local_solution = starting_solution
    best_solution_cost = calcCostforSolution(graph,starting_solution,starting_time,source)
    best_local_cost=best_solution_cost
    print(best_solution_cost)
    current_solution = starting_solution
    
    licz=0
    printSolution(graph,starting_solution,starting_time,source)
    print("------------------------------------------------------------")
    while k<STEP_LIMIT:
        #always n*(n-1) neighbours
        
        best_local_cost=1e7

        for i in range(len(stops)-1):#last stop was already switched by every previous stop
                
                stop_to_switch_1 = current_solution[i]
                for j in range(i+1,len(stops)):
                    
                        stop_to_switch_2 = current_solution[j]
                        neighbour_permutation=deepcopy(current_solution)
                        neighbour_permutation[i]=stop_to_switch_2
                        neighbour_permutation[j]=stop_to_switch_1
                        licz+=1
                        #print(neighbour_permutation)
                        if tuple(neighbour_permutation) not in T:
                            T.add(tuple(neighbour_permutation))
                            solution_cost=calcCostforSolution(graph,neighbour_permutation, starting_time,source) 
                            #print(solution_cost)
                            if solution_cost<best_local_cost:
                                best_local_solution=neighbour_permutation
                                best_local_cost=solution_cost
                                
        if best_local_cost==1e7:
             break
        
        if best_local_cost<best_solution_cost:
            best_known_solution=best_local_solution
            best_solution_cost=best_local_cost
        print(best_local_cost)
    
        current_solution=best_local_solution    
        k+=1

    print("------------------------------------------------------------")
    printSolution(graph,best_known_solution,starting_time,source)

                           

    print(best_solution_cost)



In [177]:
travellingSalesmanUsingTabuSearch(wroclaw_graph,"Tyrmanda",["most Grunwaldzki","FAT","Zamkowa","Sowia","Pereca","Klimasa"],740)

153
Final destination:  FAT  reached at  12 : 31 

Ostrowskiego - 119 - 12 : 29
Końcowa - 119 - 12 : 27
Krzemieniecka - 119 - 12 : 26
Trawowa - 119 - 12 : 25
Stanisławowska (W.K. Formaty) - 119 - 12 : 24
Muchobór Wielki - 119 - 12 : 23
Zagony - 119 - 12 : 22
Tyrmanda - 119 - 12 : 21
You are starting from:  Tyrmanda at:  12 : 20
Final destination:  Klimasa  reached at  12 : 55 

Transbud - 8 - 12 : 53
Nyska - 8 - 12 : 52
Bardzka - 8 - 12 : 51

Bardzka reached at 12:50 You have to take next bus/tram now!

Orzechowa - 133 - 12 : 48
ROD Bajki - 133 - 12 : 47

ROD Bajki reached at 12:47 You have to take next bus/tram now!

SPISKA (Ośrodek sportu) - 136 - 12 : 45
Wiśniowa - 136 - 12 : 44
Sudecka - 136 - 12 : 42
Hallera - 136 - 12 : 41
Gajowicka - 136 - 12 : 38
Mielecka - 136 - 12 : 36
Ojca Beyzyma - 136 - 12 : 34

Ojca Beyzyma reached at 12:34 You have to take next bus/tram now!

Aleja Pracy - 119 - 12 : 33
FAT - 119 - 12 : 31
You are starting from:  FAT at:  12 : 31
Final destination:  most